[Reference](https://towardsdatascience.com/blazing-hot-python-asyncio-pipelines-438b34bed9f)

In [3]:
import time

def func(targets, task_id, param=None):
    print(f"{task_id}: Initialised with param: {param}")
    while True:
        inpt = (yield)
        print(f"{task_id}: Received input: {inpt}")
        time.sleep(1)  # simulated IO delay
        for target in targets:
            print(f"{task_id}: T1 sending {inpt}")
            target.send(inpt)

gen2 = func([], 'T2',  "hello T2")
gen2.send(None)

gen1 = func([gen2], 'T1',  "hello T1")
gen1.send(None)

start_time = time.time()
gen1.send(1)
gen1.send(2)
gen1.send(3)
print(f"Duration: {time.time() - start_time}")

T2: Initialised with param: hello T2
T1: Initialised with param: hello T1
T1: Received input: 1
T1: T1 sending 1
T2: Received input: 1
T1: Received input: 2
T1: T1 sending 2
T2: Received input: 2
T1: Received input: 3
T1: T1 sending 3
T2: Received input: 3
Duration: 6.00875449180603


In [2]:
import asyncio
import time

tasks = []

def func(targets, task_id, param):
    print(f"{task_id}: Initialised with param: {param}")

    async def func_inner(targets, inpt):
        await asyncio.sleep(1)  # simulated IO delay
        print(f"{task_id}: Received input: {inpt}")
        outp = inpt
        for target in targets or []:
            print(f"{task_id}: T1 sending {outp}")
            target.send(outp)

    while True:
        inpt = (yield)
        print(f'{task_id}: Creating task with {task_id}_inner, input {inpt}.')
        tasks.append(asyncio.create_task(func_inner(targets, inpt)))


async def main():
    gen2 = func([], 'T2', "hello T2")
    gen2.send(None)

    gen1 = func([gen2], 'T1', "hello T1")
    gen1.send(None)

    start_time = time.time()
    gen1.send(1)
    gen1.send(2)
    gen1.send(3)

    await asyncio.gather(*tasks)
    await asyncio.gather(*tasks)  # round 2
    print(f"Duration: {time.time() - start_time}")

asyncio.run(main())

RuntimeError: ignored

In [4]:
import asyncio
import time

tasks = []

async def func(input_q, target_qs, task_id, param):
    print(f"{task_id}: Initialised with param: {param}")

    async def func_inner(input_q, target_qs, inpt):
        print(f"{task_id}: Recieved input: {inpt}")
        await asyncio.sleep(1)  # simulated IO delay
        outp = inpt
        for target_q in target_qs or []:
            print(f"{task_id}: T1 sending {outp}")
            await target_q.put(outp)
        input_q.task_done()

    while True:
        inpt = await input_q.get()
        print(f'{task_id}: Creating task with {task_id}_inner, input {inpt}.')
        tasks.append(asyncio.create_task(func_inner(input_q, target_qs, inpt)))


async def main():
    q2 = asyncio.Queue()
    coro2 = func(q2, [], 'T2', "hello T2")
    asyncio.create_task(coro2)

    q1 = asyncio.Queue()
    coro1 = func(q1, [q2], 'T1', "hello T1")
    asyncio.create_task(coro1)

    start_time = time.time()
    await q1.put(1)
    await q1.put(2)
    await q1.put(3)

    await q1.join()
    await q2.join()
    await asyncio.gather(*tasks)
    print(f"Duration: {time.time() - start_time}")

asyncio.run(main())

RuntimeError: ignored